In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import logging
from sklearn.neighbors import NearestNeighbors
import os

# Configuración de logging
logging.basicConfig(
  filename='procesamiento_datos.log',
  level=logging.INFO,
  format='%(asctime)s - %(levelname)s - %(message)s'
)

In [2]:
# Lectura de archivos Excel
try:
    estacionados_camion = pd.read_excel('../Limpia/estacionados_camion.xlsx')
    df_tareas = pd.read_excel('../Limpia/Tareas-limpio.xlsx')
    ubi_cliente = pd.read_excel('../Limpia/Ubicaciones_direcciones.xlsx')
    print("Archivos Excel leídos correctamente.")
except Exception as e:
    print(f"Error al leer los archivos Excel: {e}")
    exit()

Archivos Excel leídos correctamente.


In [3]:
print("______ INFO DF_TAREAS ______ ")
print("           ")
df_tareas.info()
print("           ")
print("______ INFO UBI_CLIENTE ______ ")
print("           ")
ubi_cliente.info()
print("           ")
print("______ INFO ESTACIONADOS_CAMION ______ ")
print("           ")
estacionados_camion.info()

______ INFO DF_TAREAS ______ 
           
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1286 entries, 0 to 1285
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CODIGO    1286 non-null   int64 
 1   PROYECTO  1286 non-null   object
dtypes: int64(1), object(1)
memory usage: 20.2+ KB
           
______ INFO UBI_CLIENTE ______ 
           
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CODIGO     512 non-null    int64         
 1   NOMCLI     512 non-null    object        
 2   LATITUD    512 non-null    float64       
 3   UBICACIÓN  512 non-null    object        
 4   LONGITUD   512 non-null    float64       
 5   FECHA      512 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 24.1+ KB
           
______ INFO ESTACIO

In [4]:
estacionados_camion.tail(5)

,Indice,Numero_de_placa,Estado_de_viaje,Tiempo_de_Inicio,Tiempo_Final,Duracion,Lugar_de_inicio,camion_x,camion_y
7054,118,BYD1004,Estacionamiento,2025-02-04 17:18:46,2025-02-05 08:46:51,928.08,"34.771365S,55.757825W",-34.771365,-55.757825
7055,120,BYD1004,Estacionamiento,2025-02-05 08:52:48,2025-02-05 09:07:22,14.57,"34.764805S,55.722469W",-34.764805,-55.722469
7056,122,BYD1004,Estacionamiento,2025-02-05 09:12:11,2025-02-05 09:38:27,26.27,"34.765137S,55.755452W",-34.765137,-55.755452
7057,124,BYD1004,Estacionamiento,2025-02-05 09:44:38,2025-02-05 10:04:46,20.13,"34.733343S,55.769829W",-34.733343,-55.769829
7058,126,BYD1004,Estacionamiento,2025-02-05 10:08:41,2025-02-05 10:22:48,14.12,"34.736249S,55.765682W",-34.736249,-55.765682


## === INICIO DE VERIFICACIÓN DE COORDENADAS ===
### Clientes y Camiones

### 1_ Clientes:

In [5]:
print("\n=== INICIO DE VERIFICACIÓN DE COORDENADAS ===")
print(f"Total de registros iniciales:")
print(f"Clientes: {len(ubi_cliente)}")
print(f"Camiones: {len(estacionados_camion)}")

print("\n1. VERIFICACIÓN DE COORDENADAS DE CLIENTES")
print("-------------------------------------------")
print("Criterios de validación:")
print("- Latitud: entre -90 y 90")
print("- Longitud: entre -180 y 180")
print("- ✅ Sin valores nulos")

# Verificación de clientes
coordenadas_ubi_cliente_validas = ubi_cliente[
  ubi_cliente['LATITUD'].between(-90, 90) &
  ubi_cliente['LONGITUD'].between(-180, 180) &
  ubi_cliente['LATITUD'].notnull() &
  ubi_cliente['LONGITUD'].notnull()
].copy()

coordenadas_ubi_cliente_invalidas = ubi_cliente[
  ~(
      ubi_cliente['LATITUD'].between(-90, 90) &
      ubi_cliente['LONGITUD'].between(-180, 180) &
      ubi_cliente['LATITUD'].notnull() &
      ubi_cliente['LONGITUD'].notnull()
  )
].copy()

print("\nResultados de verificación de clientes:")
print(f"✅ Clientes con coordenadas válidas: {len(coordenadas_ubi_cliente_validas)}")
print(f"❌ Clientes con coordenadas inválidas: {len(coordenadas_ubi_cliente_invalidas)}")

if not coordenadas_ubi_cliente_invalidas.empty:
  print("\n ❌ Detalle de clientes con coordenadas inválidas:")
  for _, row in coordenadas_ubi_cliente_invalidas[['CODIGO', 'LATITUD', 'LONGITUD']].iterrows():
      print(f"CODIGO: {row['CODIGO']}, LAT: {row['LATITUD']}, LON: {row['LONGITUD']}")

print("\n2. VERIFICACIÓN DE COORDENADAS DE CAMIONES")
print("-------------------------------------------")
print("Criterios de validación:")
print("- Coordenada X: entre -90 y 90")
print("- Coordenada Y: entre -180 y 180")
print("- Sin valores nulos")




=== INICIO DE VERIFICACIÓN DE COORDENADAS ===
Total de registros iniciales:
Clientes: 512
Camiones: 7059

1. VERIFICACIÓN DE COORDENADAS DE CLIENTES
-------------------------------------------
Criterios de validación:
- Latitud: entre -90 y 90
- Longitud: entre -180 y 180
- ✅ Sin valores nulos

Resultados de verificación de clientes:
✅ Clientes con coordenadas válidas: 512
❌ Clientes con coordenadas inválidas: 0

2. VERIFICACIÓN DE COORDENADAS DE CAMIONES
-------------------------------------------
Criterios de validación:
- Coordenada X: entre -90 y 90
- Coordenada Y: entre -180 y 180
- Sin valores nulos


### 2_ Camiones:

In [6]:
# Verificación de camiones
coordenadas_camion_validas = estacionados_camion[
  estacionados_camion['camion_x'].between(-90, 90) &
  estacionados_camion['camion_y'].between(-180, 180) &
  estacionados_camion['camion_x'].notnull() &
  estacionados_camion['camion_y'].notnull()
].copy()

coordenadas_camion_invalidas = estacionados_camion[
  ~(
      estacionados_camion['camion_x'].between(-90, 90) &
      estacionados_camion['camion_y'].between(-180, 180) &
      estacionados_camion['camion_x'].notnull() &
      estacionados_camion['camion_y'].notnull()
  )
].copy()

print("\nResultados de verificación de camiones:")
print(f"✅ Camiones con coordenadas válidas: {len(coordenadas_camion_validas)}")
print(f"❌ Camiones con coordenadas inválidas: {len(coordenadas_camion_invalidas)}")

if not coordenadas_camion_invalidas.empty:
  print("\nDetalle de camiones con coordenadas inválidas:")
  for _, row in coordenadas_camion_invalidas[['Numero_de_placa', 'camion_x', 'camion_y']].iterrows():
      print(f"Placa: {row['Numero_de_placa']}, X: {row['camion_x']}, Y: {row['camion_y']}")

print("\n=== FIN DE VERIFICACIÓN DE COORDENADAS ===")


Resultados de verificación de camiones:
✅ Camiones con coordenadas válidas: 7059
❌ Camiones con coordenadas inválidas: 0

=== FIN DE VERIFICACIÓN DE COORDENADAS ===


## === INICIO DE ASIGNACIÓN DE CLIENTES A CAMIONES ===
### Asignación de Cliente más Cercano

In [7]:
print("\n=== INICIO DE ASIGNACIÓN DE CLIENTES A CAMIONES ===")

print("\n1. PREPARANDO DATOS PARA EL MODELO")
print("-----------------------------------")
X = coordenadas_ubi_cliente_validas[['LATITUD', 'LONGITUD']].values
y = coordenadas_camion_validas[['camion_x', 'camion_y']].values
print(f"Matriz de coordenadas de clientes (X): {X.shape}")
print(f"Matriz de coordenadas de camiones (y): {y.shape}")

if X.shape[0] == 0 or y.shape[0] == 0:
  print("❌ No hay suficientes datos con coordenadas válidas")
else:
  print("\n2. ENTRENANDO MODELO NEAREST NEIGHBORS")
  print("--------------------------------------")
  print("Configuración del modelo:")
  print("- n_neighbors: 1")
  print("- algorithm: auto")
  print("- metric: euclidean")
  print("- p: 2 (norma L2)")
  
  nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto', metric='euclidean', p=2).fit(X)
  print("✓ Modelo entrenado exitosamente")

  print("\n3. CALCULANDO DISTANCIAS Y ASIGNANDO CLIENTES")
  print("--------------------------------------------")
  distances, indices = nbrs.kneighbors(y)
  
  # Estadísticas de las distancias
  print("\nEstadísticas de distancias (en grados):")
  print(f"- Distancia mínima: {distances.min():.4f}°")
  print(f"- Distancia máxima: {distances.max():.4f}°")
  print(f"- Distancia promedio: {distances.mean():.4f}°")
  print(f"- Desviación estándar: {distances.std():.4f}°")
  
  # Distribución de distancias
  print("\nDistribución de distancias:")
  percentiles = [25, 50, 75]
  for p in percentiles:
      print(f"- Percentil {p}: {np.percentile(distances, p):.4f}°")
  
  # Conteo de asignaciones
  print(f"\nTotal de asignaciones realizadas: {len(indices)}")
  
  # Análisis de clientes asignados
  clientes_asignados = coordenadas_ubi_cliente_validas.iloc[indices.flatten()]['CODIGO'].value_counts()
  print("\nEstadísticas de asignación de clientes:")
  print(f"- Clientes únicos asignados: {len(clientes_asignados)}")
  if len(clientes_asignados) > 0:
      print(f"- Máximo de veces que se asignó un mismo cliente: {clientes_asignados.max()}")
      print("\nTop 5 clientes más asignados:")
      for codigo, count in clientes_asignados.head().items():
          print(f"  Cliente {codigo}: {count} veces")

  print("\n4. ASIGNANDO CÓDIGOS DE CLIENTES A CAMIONES")
  print("------------------------------------------")
  coordenadas_camion_validas['CODIGO'] = coordenadas_ubi_cliente_validas.iloc[indices.flatten()]['CODIGO'].values
  print("✓ Códigos asignados exitosamente")
  
  # Añadir distancias al DataFrame
  coordenadas_camion_validas['distancia_al_cliente'] = distances.flatten()
  print("\nEstadísticas de asignaciones:")
  print(f"- Asignaciones a menos de 1°: {(distances < 1).sum()} ({(distances < 1).sum()/len(distances)*100:.1f}%)")
  print(f"- Asignaciones a menos de 5°: {(distances < 5).sum()} ({(distances < 5).sum()/len(distances)*100:.1f}%)")
  print(f"- Asignaciones a menos de 10°: {(distances < 10).sum()} ({(distances < 10).sum()/len(distances)*100:.1f}%)")

  print("\n5. COMBINANDO RESULTADOS FINALES")
  print("--------------------------------")
  estacionados_camion_final = pd.concat([coordenadas_camion_validas, coordenadas_camion_invalidas], ignore_index=True)
  estacionados_camion_final = estacionados_camion_final.sort_index()
  print(f"Total de registros en resultado final: {len(estacionados_camion_final)}")
  print(f"- Camiones con cliente asignado: {len(coordenadas_camion_validas)}")
  print(f"- Camiones sin asignación: {len(coordenadas_camion_invalidas)}")
  
  # Resumen final de calidad de asignaciones
  print("\nResumen de calidad de asignaciones:")
  print(f"- Porcentaje de camiones con asignación: {len(coordenadas_camion_validas)/len(estacionados_camion_final)*100:.1f}%")
  print(f"- Distancia promedio a clientes: {distances.mean():.4f}°")
  print(f"- Mediana de distancia a clientes: {np.median(distances):.4f}°")

print("\n=== FIN DE ASIGNACIÓN DE CLIENTES A CAMIONES ===")


=== INICIO DE ASIGNACIÓN DE CLIENTES A CAMIONES ===

1. PREPARANDO DATOS PARA EL MODELO
-----------------------------------
Matriz de coordenadas de clientes (X): (512, 2)
Matriz de coordenadas de camiones (y): (7059, 2)

2. ENTRENANDO MODELO NEAREST NEIGHBORS
--------------------------------------
Configuración del modelo:
- n_neighbors: 1
- algorithm: auto
- metric: euclidean
- p: 2 (norma L2)
✓ Modelo entrenado exitosamente

3. CALCULANDO DISTANCIAS Y ASIGNANDO CLIENTES
--------------------------------------------

Estadísticas de distancias (en grados):
- Distancia mínima: 0.0000°
- Distancia máxima: 1.1330°
- Distancia promedio: 0.0017°
- Desviación estándar: 0.0299°

Distribución de distancias:
- Percentil 25: 0.0001°
- Percentil 50: 0.0003°
- Percentil 75: 0.0007°

Total de asignaciones realizadas: 7059

Estadísticas de asignación de clientes:
- Clientes únicos asignados: 392
- Máximo de veces que se asignó un mismo cliente: 509

Top 5 clientes más asignados:
  Cliente 80283: 5

## === VERIFICACION DE DF TAREAS CODIGO  ===

In [8]:
print("\n=== INICIO DE LIMPIEZA DE DATOS ===")

# Procesamiento de df_tareas
print("\n1. PROCESANDO DF_TAREAS")
print("----------------------")
print("Verificando y limpiando códigos...")

# Identificar registros inválidos
df_tareas_invalidas = df_tareas[
  df_tareas['CODIGO'].isnull() | 
  (df_tareas['CODIGO'].apply(lambda x: not str(x).isdigit()))
].copy()
print(f"Filas con 'CODIGO' nulo o no numérico: {df_tareas_invalidas.shape[0]}")

# Limpiar y convertir CODIGO
df_tareas = df_tareas.dropna(subset=['CODIGO']).copy()
df_tareas['CODIGO'] = pd.to_numeric(df_tareas['CODIGO'], errors='coerce')
filas_iniciales = df_tareas.shape[0]
df_tareas = df_tareas.dropna(subset=['CODIGO']).copy()
filas_finales = df_tareas.shape[0]
df_tareas['CODIGO'] = df_tareas['CODIGO'].astype(int)

print(f"Filas eliminadas: {filas_iniciales - filas_finales}")
print(f"Filas restantes: {filas_finales}")

# Filtrar columnas necesarias
df_tareas_filtrado = df_tareas[['CODIGO', 'PROYECTO']].copy()
print(f"Columnas seleccionadas: {list(df_tareas_filtrado.columns)}")

print("\n2. PROCESANDO ESTACIONADOS_CAMION")
print("--------------------------------")
# Convertir columnas de tiempo
for columna in ['Tiempo_de_Inicio', 'Tiempo_Final']:
  if columna in estacionados_camion.columns:
      estacionados_camion[columna] = pd.to_datetime(estacionados_camion[columna], errors='coerce')
      print(f"✓ Columna '{columna}' convertida a datetime")

# Filtrar camiones estacionados
if 'Estado_de_viaje' in estacionados_camion.columns:
  total_camiones = estacionados_camion.shape[0]
  estacionados_camion_filtered = estacionados_camion[
      estacionados_camion['Estado_de_viaje'] == 'Estacionamiento'
  ].copy()
  print(f"Camiones estacionados: {estacionados_camion_filtered.shape[0]} de {total_camiones}")
else:
  print("❌ Error: Falta columna 'Estado_de_viaje'")
  raise ValueError("Columna 'Estado_de_viaje' no encontrada")

print("\n=== FIN DE LIMPIEZA DE DATOS ===")


=== INICIO DE LIMPIEZA DE DATOS ===

1. PROCESANDO DF_TAREAS
----------------------
Verificando y limpiando códigos...
Filas con 'CODIGO' nulo o no numérico: 0
Filas eliminadas: 0
Filas restantes: 1286
Columnas seleccionadas: ['CODIGO', 'PROYECTO']

2. PROCESANDO ESTACIONADOS_CAMION
--------------------------------
✓ Columna 'Tiempo_de_Inicio' convertida a datetime
✓ Columna 'Tiempo_Final' convertida a datetime
Camiones estacionados: 7059 de 7059

=== FIN DE LIMPIEZA DE DATOS ===


In [9]:

print("\n2. PROCESANDO ESTACIONADOS_CAMION")
print("--------------------------------")
# Convertir columnas de tiempo
for columna in ['Tiempo_de_Inicio', 'Tiempo_Final']:
  if columna in estacionados_camion.columns:
      estacionados_camion[columna] = pd.to_datetime(estacionados_camion[columna], errors='coerce')
      print(f"✓ Columna '{columna}' convertida a datetime")

# Filtrar camiones estacionados
if 'Estado_de_viaje' in estacionados_camion.columns:
  total_camiones = estacionados_camion.shape[0]
  estacionados_camion_filtered = estacionados_camion[
      estacionados_camion['Estado_de_viaje'] == 'Estacionamiento'
  ].copy()
  print(f"Camiones estacionados: {estacionados_camion_filtered.shape[0]} de {total_camiones}")
else:
  print("❌ Error: Falta columna 'Estado_de_viaje'")
  raise ValueError("Columna 'Estado_de_viaje' no encontrada")

print("\n=== FIN DE LIMPIEZA DE DATOS ===")


2. PROCESANDO ESTACIONADOS_CAMION
--------------------------------
✓ Columna 'Tiempo_de_Inicio' convertida a datetime
✓ Columna 'Tiempo_Final' convertida a datetime
Camiones estacionados: 7059 de 7059

=== FIN DE LIMPIEZA DE DATOS ===


## === INICIO DE PROCESO DE MERGES ===

In [10]:
print("\n=== INICIO DE PROCESO DE MERGES ===")

print("\n1. PRIMER MERGE: df_tareas_filtrado con ubi_cliente")
print("---------------------------------------------")
print(f"Estado inicial:")
print(f"- df_tareas_filtrado: {df_tareas_filtrado.shape} filas")
print(f"- ubi_cliente: {ubi_cliente.shape} filas")

# Primer merge
merged_df = pd.merge(df_tareas_filtrado, ubi_cliente, on='CODIGO', how='inner')
print("\nResultado del primer merge:")
print(f"- Filas resultantes: {merged_df.shape[0]}")
print(f"- Columnas resultantes: {merged_df.shape[1]}")
print(f"- Columnas: {merged_df.columns.tolist()}")

print("\n2. MERGE FINAL")
print("------------")
print("Realizando merge final entre merged_df y estacionados_camion_final")
# Usamos estacionados_camion_final que ya tiene la columna CODIGO asignada
merged_final_df = pd.merge(
  merged_df, 
  estacionados_camion_final,  # Usamos el DataFrame que ya tiene CODIGO
  on='CODIGO', 
  how='inner',
  suffixes=('_tareas', '_camion')
)
print(f"\nResultado del merge final:")
print(f"- Filas: {merged_final_df.shape[0]}")
print(f"- Columnas: {merged_final_df.shape[1]}")

print("\n3. SELECCIÓN DE COLUMNAS FINALES")
print("------------------------------")
columnas_finales = [
  'CODIGO', 'PROYECTO', 'NOMCLI', 'Duracion', 'Tiempo_de_Inicio', 'UBICACIÓN',
  'Numero_de_placa', 'LATITUD', 'LONGITUD', 'camion_x', 'camion_y'
]

# Verificar columnas disponibles
columnas_faltantes = [col for col in columnas_finales if col not in merged_final_df.columns]
if columnas_faltantes:
  print("\n⚠️ Columnas faltantes:")
  for col in columnas_faltantes:
      print(f"- {col}")
else:
  filtered_df = merged_final_df[columnas_finales].copy()
  print("\nColumnas seleccionadas exitosamente:")
  for col in columnas_finales:
      print(f"- {col}")



=== INICIO DE PROCESO DE MERGES ===

1. PRIMER MERGE: df_tareas_filtrado con ubi_cliente
---------------------------------------------
Estado inicial:
- df_tareas_filtrado: (1286, 2) filas
- ubi_cliente: (512, 6) filas

Resultado del primer merge:
- Filas resultantes: 1139
- Columnas resultantes: 7
- Columnas: ['CODIGO', 'PROYECTO', 'NOMCLI', 'LATITUD', 'UBICACIÓN', 'LONGITUD', 'FECHA']

2. MERGE FINAL
------------
Realizando merge final entre merged_df y estacionados_camion_final

Resultado del merge final:
- Filas: 21047
- Columnas: 17

3. SELECCIÓN DE COLUMNAS FINALES
------------------------------

Columnas seleccionadas exitosamente:
- CODIGO
- PROYECTO
- NOMCLI
- Duracion
- Tiempo_de_Inicio
- UBICACIÓN
- Numero_de_placa
- LATITUD
- LONGITUD
- camion_x
- camion_y


## GUARDANDO RESULTADO

In [11]:

print("\n4. GUARDANDO RESULTADO")
print("--------------------")
try:
  filtered_df.to_excel('../Limpia/merged_df.xlsx', index=False)
  print("✓ Archivo guardado exitosamente")
  
  print("\nVerificación final de nulos:")
  nulos = filtered_df.isnull().sum()
  for columna, cantidad in nulos.items():
      print(f"- {columna}: {cantidad} nulos")
except Exception as e:
  print(f"❌ Error al guardar: {e}")

print("\n=== FIN DE PROCESO DE MERGES ===")


4. GUARDANDO RESULTADO
--------------------
✓ Archivo guardado exitosamente

Verificación final de nulos:
- CODIGO: 0 nulos
- PROYECTO: 0 nulos
- NOMCLI: 0 nulos
- Duracion: 0 nulos
- Tiempo_de_Inicio: 0 nulos
- UBICACIÓN: 0 nulos
- Numero_de_placa: 0 nulos
- LATITUD: 0 nulos
- LONGITUD: 0 nulos
- camion_x: 0 nulos
- camion_y: 0 nulos

=== FIN DE PROCESO DE MERGES ===


In [12]:
filtered_df.info()
filtered_df.tail(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21047 entries, 0 to 21046
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CODIGO            21047 non-null  int32         
 1   PROYECTO          21047 non-null  object        
 2   NOMCLI            21047 non-null  object        
 3   Duracion          21047 non-null  float64       
 4   Tiempo_de_Inicio  21047 non-null  datetime64[ns]
 5   UBICACIÓN         21047 non-null  object        
 6   Numero_de_placa   21047 non-null  object        
 7   LATITUD           21047 non-null  float64       
 8   LONGITUD          21047 non-null  float64       
 9   camion_x          21047 non-null  float64       
 10  camion_y          21047 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int32(1), object(4)
memory usage: 1.8+ MB


,CODIGO,PROYECTO,NOMCLI,Duracion,Tiempo_de_Inicio,UBICACIÓN,Numero_de_placa,LATITUD,LONGITUD,camion_x,camion_y
21042,80240,💲Creditos Dario,KIOSCO INDERUT,8.42,2024-12-27 14:20:20,Marcador 70\n2024/10/28 @ 13:20:01\nhttp://map...,PEUGEOTAVG9758,-34.783782,-55.495198,-34.766024,-55.495940
21043,80240,💲Creditos Dario,KIOSCO INDERUT,15.70,2025-01-08 13:00:00,Marcador 70\n2024/10/28 @ 13:20:01\nhttp://map...,PEUGEOTAVG9758,-34.783782,-55.495198,-34.766047,-55.495940
21044,80240,💲Creditos Dario,KIOSCO INDERUT,6.05,2025-01-20 14:40:13,Marcador 70\n2024/10/28 @ 13:20:01\nhttp://map...,PEUGEOTAVG9758,-34.783782,-55.495198,-34.766047,-55.495955
21045,80240,💲Creditos Dario,KIOSCO INDERUT,9.88,2025-01-27 15:40:56,Marcador 70\n2024/10/28 @ 13:20:01\nhttp://map...,PEUGEOTAVG9758,-34.783782,-55.495198,-34.765997,-55.495960
21046,80240,💲Creditos Dario,KIOSCO INDERUT,8.65,2025-01-29 12:31:44,Marcador 70\n2024/10/28 @ 13:20:01\nhttp://map...,PEUGEOTAVG9758,-34.783782,-55.495198,-34.766094,-55.495899
